# Mapping Hi-C data with `bwa` and `pairsamtools`

In [ ]:
import os

if os.path.exists('mapping'):
    !rm -rf mapping

In [ ]:
!mkdir -p mapping

In [ ]:
cd mapping

### Download sample yeast reads

In [ ]:
%%bash

curl -LkSs https://api.github.com/repos/mirnylab/distiller-test-data/tarball | tar -zxf - 
mv $(ls -d mirnylab-distiller-test-data-* | head -n 1)/genome .
mv $(ls -d mirnylab-distiller-test-data-* | head -n 1)/fastq .

In [ ]:
ll genome/

In [ ]:
ll fastq/*/*

### Simple pipeline for a single run or part of a run

In [ ]:
%%bash

set -o errexit
set -o nounset
set -o pipefail

INDEX='./genome/sacCer3.fa.gz'
CHROMSIZES='./genome/sacCer3.chrom.sizes'
FASTQ1='./fastq/MATalpha_R1/lane1/SRR2601842_1.fastq.gz'
FASTQ2='./fastq/MATalpha_R1/lane1/SRR2601842_1.fastq.gz'
OUTPREFIX='MATalpha_R1'

N_THREADS=8

UNMAPPED_SAM_PATH=${OUTPREFIX}.unmapped.bam
UNMAPPED_PAIRS_PATH=${OUTPREFIX}.unmapped.pairs.gz
NODUPS_SAM_PATH=${OUTPREFIX}.nodups.bam
NODUPS_PAIRS_PATH=${OUTPREFIX}.nodups.pairs.gz
DUPS_SAM_PATH=${OUTPREFIX}.dups.bam
DUPS_PAIRS_PATH=${OUTPREFIX}.dups.pairs.gz

bwa mem -SPM -t "${N_THREADS}" "${INDEX}" "${FASTQ1}" "${FASTQ2}" | {
    # Classify Hi-C molecules as unmapped/single-sided/multimapped/chimeric/etc
    # and output one line per read, containing the following, separated by \\v:
    #  * triu-flipped pairs
    #  * read id
    #  * type of a Hi-C molecule
    #  * corresponding sam entries
    pairsamtools parse -c ${CHROMSIZES}
} | {
    # Block-sort pairs together with SAM entries
    pairsamtools sort
} | {
    # Set unmapped and ambiguous reads aside
    pairsamtools select '(pair_type == "CX") or (pair_type == "LL")' \
        --output-rest >( pairsamtools split \
            --output-pairs ${UNMAPPED_PAIRS_PATH} \
            --output-sam ${UNMAPPED_SAM_PATH} ) 
} | {
    # Remove duplicates
    pairsamtools dedup \
        --output \
            >( pairsamtools split \
                --output-pairs ${NODUPS_PAIRS_PATH} \
                --output-sam ${NODUPS_SAM_PATH} ) \
        --output-dups \
            >( pairsamtools markasdup \
                | pairsamtools split \
                    --output-pairs ${DUPS_PAIRS_PATH} \
                    --output-sam ${DUPS_SAM_PATH} )
}

In [ ]:
ll

In [ ]:
!samtools view MATalpha_R1.nodups.bam | head

In [ ]:
!samtools view MATalpha_R1.dups.bam | head

In [ ]:
!zcat MATalpha_R1.nodups.pairs.gz | head -n 50

### Index the deduped pairs file

In [ ]:
!pairix MATalpha_R1.nodups.pairs.gz

In [ ]:
ll

In [ ]:
!pairix MATalpha_R1.nodups.pairs.gz 'chrI|chrI'